In [27]:
import cv2
import numpy as np
import pandas as pd
images = []
for dirname, _, filenames in os.walk('classification'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname, filename))
        images.append(img)
images = np.asarray(images)

In [28]:
labels = []
for dirname, _, filenames in os.walk('csvs'):
    for filename in filenames:
        labs = pd.read_csv(os.path.join(dirname, filename))['label'].dropna().values
        labels.append(labs)
labels = np.asarray(labels).reshape(-1)

In [29]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
torch.cuda.is_available()

True

In [66]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    # input_size:28, same_padding=(filter_size-1)/2, 3-1/2=1:padding
    self.cnn1=nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
    # input_size-filter_size +2(padding)/stride + 1 = 28-3+2(1)/1+1=28
    self.batchnorm1=nn.BatchNorm2d(8)
    # output_channel:8, batch(8)
    self.relu=nn.ReLU()
    self.maxpool1=nn.MaxPool2d(kernel_size=2)
    #input_size=28/2=14
    self.cnn2=nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
    # same_padding: (5-1)/2=2:padding_size. 
    self.batchnorm2=nn.BatchNorm2d(32)
    self.maxpool2=nn.MaxPool2d(kernel_size=2)
    # input_size=14/2=7
    # 32x7x7=1568
    self.fc1 =nn.Linear(in_features=32*32*32, out_features=600)
    self.dropout= nn.Dropout(p=0.5)
    self.fc2 =nn.Linear(in_features=600, out_features=2)
  def forward(self,x):
    out =self.cnn1(x)
    out =self.batchnorm1(out)
    out =self.relu(out)
    out =self.maxpool1(out)
    out =self.cnn2(out)
    out =self.batchnorm2(out)
    out =self.relu(out)
    out =self.maxpool2(out)
    out =out.view(64,32*32*32)
    out =self.fc1(out)
    out =self.relu(out)
    out =self.dropout(out)
    out =self.fc2(out)
    return out

In [67]:
model=CNN()
CUDA=torch.cuda.is_available()
if CUDA:
  model=model.cuda()
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=0.01)

In [94]:
import torchvision
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, images, labels):
        'Initialization'
        self.labels = labels
        self.images = images / 255.0

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.images)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample

        return self.images[index], self.labels[index]

In [95]:
# Generators
from torch.utils import data

# Parameters
params = {'batch_size': 64,
          'shuffle': True,
         'drop_last': True}

training_set = Dataset(images, labels)
training_generator = data.DataLoader(training_set, **params)

In [96]:
iteration=0
epochs=1000
for epoch in range(epochs):
  for i, (images,labels) in enumerate(training_generator):
    iteration+=1
    if CUDA:
      images =Variable(images.cuda())
      labels =Variable(labels.cuda())
    else:
      images =Variable(images)
      labels =Variable(labels)
      
    optimizer.zero_grad()
    outputs=model(images)
    loss=loss_function(outputs,labels)
    loss.backward()
    optimizer.step()
    
    if(iteration+1)%100 ==0:
        correct =0
        total =0
        for images,labels in training_generator:
            if CUDA:
              images =Variable(images.cuda())
            else:
              images =Variable(images)

            outputs=model(images)
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            if CUDA:
              correct += (predicted.cpu()==labels.cpu()).sum()
            else:
              correct += (predicted==labels).sum()

        accuracy = 100 *correct/total
        print("Iteration: {}, Train Loss: {}, Test Accuracy:{}%".format(iteration, loss.item(),accuracy))
        
print("Finished!")
    

Iteration: 99, Train Loss: 0.4839556813240051, Test Accuracy:42%
Iteration: 199, Train Loss: 0.5545804500579834, Test Accuracy:42%


KeyboardInterrupt: 